In [1]:
import requests
import time
import json
import os
import datetime
import boto3
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta

In [2]:
from config import ACCESS_KEY,SECRET_KEY,TOKEN,FOLDER,FOLDER_ID,BUCKET_NAME,BUCKET_NAME

In [3]:
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow") #настройка функции
TEMP_FILENAME = "temp_file"

headers={'Authorization':TOKEN ,'Accept':'application/json'}

def get_now_datetime_str(): # получаем актуальное время
    time_zone = os.getenv("TIME_ZONE", "Europe/Moscow") # меняем таймзону на московскую
    now = datetime.datetime.now(pytz.timezone(time_zone))    
    yesterday = now - datetime.timedelta(days=1) #нужна вчерашняя дата так как данные за прошлый день
    last_month_data = now - relativedelta(month=1)
    return {'key_parquet': yesterday.strftime('year=%Y/month=%m/%d.parquet'),
            'key': yesterday.strftime('year=%Y/month=%m/%d.csv'),
            'key_month': yesterday.strftime('year=%Y/month=%m.csv'),
            'now':now.strftime('%Y-%m-%d %H:%M:%S'),
            'yesterday_data':yesterday.strftime('%Y-%m-%d'),
            'yesterday':yesterday.strftime('%Y-%m-%d %H:%M:%S'), 
            'year':yesterday.strftime('%Y'),
            'month':yesterday.strftime('%m'),
            'day':yesterday.strftime('%d'),
            'last_month_data':last_month_data.strftime('%Y-%m-%d')
            }

def create_query(): #функция создает новый запрос и возвращает id для запроса результата
    body = {
        "name":query_name, 
        "TYPE":"ANALYTICS", 
        "text":query_text, 
        "description":query_description
    }
    response = requests.post(
        f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries?project={FOLDER_ID}',
        headers=headers,
        json=body
    )
    if response.status_code == 200:
        return response.json()["id"]
    return f' Code: {response},  text: {response.text}'


def get_request(offset): # фунция возвращает ответ запроса. Максимум 1000 строк.
    offset = offset
    get_query_results_url = f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries/{request_id}/results/0?project={FOLDER_ID}&offset={str(offset)}&limit=1000'
    response = requests.get(
        get_query_results_url,
        headers = headers
    )
    return response

def if_cell_is_list(cell): # функция участвует в преобразовании данных при создании файла
    if isinstance(cell, list):
        if len(cell) == 0:
            return ''
        else: 
            return cell[0]
    else:
        return cell

def write_temp_file():
    offset = 0
    response = get_request(offset) #запрашиваем данные запроса
    columns = [rows['name'] for rows in response.json()['columns']] #выделяем названия столбцов
    special_str = ""
    for j in columns:
        special_str = f"{special_str}{str(j)},"
    temp_file = open(TEMP_FILENAME, 'w')
    temp_file.write(special_str[:-1]+'\n')

def write_temp_file():
    offset = 0
    response = get_request(offset) #запрашиваем данные запроса
    columns = [rows['name'] for rows in response.json()['columns']] #выделяем названия столбцов
    special_str = ""
    for j in columns:
        special_str = f"{special_str}{str(j)},"
    temp_file = open(TEMP_FILENAME, 'w', encoding='utf-8')
    temp_file.write(special_str[:-1]+'\n')

    while response.status_code == 200 and len(response.json()['rows']) != 0:  #Цикл делает запросы по 10000, пока не кончатся данные
        response = get_request(offset)
        response_rows = response.json()['rows']
        rows = [[if_cell_is_list(cell) for cell in row] for row in response_rows]  #Преобразуются строки
        # Открывает созданный файл и добавляет в него строки
        for row in rows:
            special_str = ','.join("'{0}'".format(i.replace("'", ""))  if isinstance(i, str) else str(i) for i in row)
            temp_file.write(special_str+'\n') 
        offset +=1000 # увеличивает смещение

def get_s3_instance(): # функция создает соединение
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )

def upload_dump_to_s3(): # функция выгружает данные в s3
    get_s3_instance().upload_file(
        Filename=TEMP_FILENAME,
        Bucket=BUCKET_NAME,
        Key=key
    )

def remove_temp_files(): #функция удаляет временный файл
    os.remove(TEMP_FILENAME)

In [4]:
s3 = get_s3_instance()

flats_st_partner

In [ ]:
BUCKET_NAME = "dwh-asgard"
FOLDER = 'flats_st_partner'
s3_file_name = f"{BUCKET_NAME}/{FOLDER}/{get_now_datetime_str()['key']}"
s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])

path_on_pc = 'E:/s3'
local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
local_download_path = f'{path_on_pc}/{s3_file_name}'

if os.path.isfile(local_download_path) is False:
# создается новая папка, если ее нет
    os.makedirs(local_download_folder, exist_ok=True)
    s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

In [ ]:
flats_st_partner_df = pd.read_csv('local_download_path',sep = ',')

In [ ]:
BUCKET_NAME = "dwh-asgard"
FOLDER = 'flats_dir_partner'
s3_file_name = f"{BUCKET_NAME}/{FOLDER}/flats_dir_partner.csv"
s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])

path_on_pc = 'E:/s3'
local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
local_download_path = f'{path_on_pc}/{s3_file_name}'

if os.path.isfile(local_download_path) is False:
# создается новая папка, если ее нет
    os.makedirs(local_download_folder, exist_ok=True)
    s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

In [ ]:
flats_dir_partner_df = pd.read_csv('local_download_path',sep = ',')

In [ ]:
flats_st_partner_df = flats_st_partner_df.merge(
    flats_dir_partner_df['flat_uuid','address_uuid'],
    on='flat_uuid'
    how='left'
)

installation_point_st_partner

In [ ]:
BUCKET_NAME = "dwh-asgard"
FOLDER = 'installation_point_st_partner'
s3_file_name = f"{BUCKET_NAME}/{FOLDER}/{get_now_datetime_str()['key']}"
s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])

path_on_pc = 'E:/s3'
local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
local_download_path = f'{path_on_pc}/{s3_file_name}'

if os.path.isfile(local_download_path) is False:
# создается новая папка, если ее нет
    os.makedirs(local_download_folder, exist_ok=True)
    s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

In [ ]:
installation_point_st_partner_df = pd.read_csv('local_download_path',sep = ',')

In [ ]:
BUCKET_NAME = "dwh-asgard"
FOLDER = 'entries_installation_points_dir_partner'
s3_file_name = f"{BUCKET_NAME}/{FOLDER}/entries_installation_points_dir_partner.csv"
s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])

path_on_pc = 'E:/s3'
local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
local_download_path = f'{path_on_pc}/{s3_file_name}'

if os.path.isfile(local_download_path) is False:
# создается новая папка, если ее нет
    os.makedirs(local_download_folder, exist_ok=True)
    s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

In [ ]:
entries_installation_points_dir_partner_df = pd.read_csv('local_download_path',sep = ',')

In [ ]:
installation_point_st_partner_df = installation_point_st_partner_df.merge(
    entries_installation_points_dir_partner_df['installation_point_id','address_uuid','city','сountry','region','parent_uuid'],
    on='installation_point_id'
    how='left'
)

entries_st_mobile

In [ ]:
BUCKET_NAME = "dwh-asgard"
FOLDER = 'entries_st_mobile'
s3_file_name = f"{BUCKET_NAME}/{FOLDER}/{get_now_datetime_str()['key']}"
s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])

path_on_pc = 'E:/s3'
local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
local_download_path = f'{path_on_pc}/{s3_file_name}'

if os.path.isfile(local_download_path) is False:
# создается новая папка, если ее нет
    os.makedirs(local_download_folder, exist_ok=True)
    s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

In [ ]:
entries_st_mobile_df = pd.read_csv('local_download_path',sep = ',')

merge

In [ ]:
flats_st_partner_df_merged = flats_st_partner_df.merge(
    installation_point_st_partner_df['installation_point_id','address_uuid','city','сountry','region','parent_uuid'],
    on='address_uuid',
    how='left'
)

flats_st_partner_df_merged=flats_st_partner_df_merged.merge(
    entries_st_mobile_df['address_uuid','monetization']
    on='address_uuid',
    how='left'
)

In [ ]:
flats_st_partner_df_merged.to_parquet(f'{path_on_pc}/{FOLDER}/{get_now_datetime_str()['key_parquet']}', compression='snappy', index=False) 

In [ ]:
BUCKET_NAME = "aggregated-data"
FOLDER = 'flats_research_dashboard'

s3_file_path = f"{FOLDER}/{get_now_datetime_str()['key_parquet']}"
pc_file_path = f"{path_on_pc}/{FOLDER}/{get_now_datetime_str()['key_parquet']}"

s3.upload_file(pc_file_path, BUCKET_NAME, s3_file_path)

